In [1]:
import urllib3
import torch
import torch.nn as nn 
import torch.nn.functional as F 
import plotly.express as px 
import numpy as np 

In [2]:
http = urllib3.PoolManager()
file_name = 'input.txt'
url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
response = http.request("GET", url=url)
if response.status == 200:
    with open(file=file_name, mode='wb') as f:
        f.write(response.data)
        print('file download successfully.')
else:
    print(f'file not downloaded. check url!')

file download successfully.


In [3]:
with open(file=file_name, mode='r', encoding='utf-8') as f:
    text = f.read()

print(len(text))
print(text[:1000])

1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for re

In [4]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
# create a mapping from characters to integers
stoi = { ch : i for i, ch in enumerate(chars) }
itos = { i : ch for i, ch in enumerate(chars) }

def encode(s):
    """takes a string, outputs list of integers"""
    return [stoi[ch] for ch in s]

def decode(l):
    """takes list of intergers, returns a string"""
    return ''.join([itos[i] for i in l])

print(encode('hii, there!'))
print(decode(encode('hii, there!')))

[46, 47, 47, 6, 1, 58, 46, 43, 56, 43, 2]
hii, there!


In [6]:
# let's now encode the entire text dataset and store it into a torch.Tensor
data = torch.tensor(encode(text))
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [7]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n] 
val_data = data[n:]

In [8]:
block_size = 8
train_data[:block_size]

tensor([18, 47, 56, 57, 58,  1, 15, 47])

In [9]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'When input is {context.tolist()}, the target is {target}')

When input is [18], the target is 47
When input is [18, 47], the target is 56
When input is [18, 47, 56], the target is 57
When input is [18, 47, 56, 57], the target is 58
When input is [18, 47, 56, 57, 58], the target is 1
When input is [18, 47, 56, 57, 58, 1], the target is 15
When input is [18, 47, 56, 57, 58, 1, 15], the target is 47
When input is [18, 47, 56, 57, 58, 1, 15, 47], the target is 58


In [10]:
block_size = 8
batch_size = 4

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    xb = torch.stack([data[i:block_size+i] for i in ix])
    yb = torch.stack([data[i+1:1+block_size+i] for i in ix])
    return xb, yb 

xb, yb = get_batch('train')
print(xb)
print(xb.shape)
print('---')

print(yb)
print(yb.shape)
print('---')

for b in range(batch_size): # batch dimension
    print(f'---Batch {b+1}---')
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'When input is {context}, the target is {target}')
    

tensor([[41, 58,  1, 51, 43,  0, 20, 53],
        [ 1, 39, 52, 42,  1, 41, 53, 52],
        [52, 53, 58, 46, 43, 56,  6,  1],
        [52, 58, 53,  1, 58, 46, 63,  1]])
torch.Size([4, 8])
---
tensor([[58,  1, 51, 43,  0, 20, 53, 61],
        [39, 52, 42,  1, 41, 53, 52, 57],
        [53, 58, 46, 43, 56,  6,  1, 44],
        [58, 53,  1, 58, 46, 63,  1, 50]])
torch.Size([4, 8])
---
---Batch 1---
When input is tensor([41]), the target is 58
When input is tensor([41, 58]), the target is 1
When input is tensor([41, 58,  1]), the target is 51
When input is tensor([41, 58,  1, 51]), the target is 43
When input is tensor([41, 58,  1, 51, 43]), the target is 0
When input is tensor([41, 58,  1, 51, 43,  0]), the target is 20
When input is tensor([41, 58,  1, 51, 43,  0, 20]), the target is 53
When input is tensor([41, 58,  1, 51, 43,  0, 20, 53]), the target is 61
---Batch 2---
When input is tensor([1]), the target is 39
When input is tensor([ 1, 39]), the target is 52
When input is tensor([ 1,

In [11]:
print(xb) #  our input to the transformer

tensor([[41, 58,  1, 51, 43,  0, 20, 53],
        [ 1, 39, 52, 42,  1, 41, 53, 52],
        [52, 53, 58, 46, 43, 56,  6,  1],
        [52, 58, 53,  1, 58, 46, 63,  1]])


In [12]:
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, target=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B, T, C)
        
        if target is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            target = target.view(B*T)
            loss = F.cross_entropy(logits, target)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the prediction
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # become (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
logits, loss = model.forward(xb, yb)
print(logits.shape)
print(loss)
print(decode(model.generate(torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))

torch.Size([32, 65])
tensor(4.5576, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJt-wBpm&yiltNCjeO3:Cx&vvMYW-txjuAd IRFbTpJ$zkZelxZtTlHNzdXXUiQQY:qFINTOBNLI,&oTigq z.c:Cq,SDXzetn3XVjX-YBcHAUhk&PHdhcOb
nhJ?FJU?pRiOLQeUN!BxjPLiq-GJdUV'hsnla!murI!IM?SPNPq?VgC'R
pD3cLv-bxn-tL!upg
SZ!Uvdg CtxtT?hsiW:XxKIiPlagHIsr'zKSVxza?GlDWObPmRJgrIAcmspmZ&viCKot:u3qYXA:rZgv f:3Q-oiwUzqh'Z!I'zRS3SP rVchSFUIdd q?sPJpUdhMCK$VXXevXJFMl,i
YxA:gWId,EXR,iMC,$?srV$VztRwb?KpgUWFjR$zChOLm;JrDnDph
LBj,KZxJa


In [13]:
block_size = 3
batch_size = 2

xb_new, yb_new = get_batch('train')
print(xb_new)
print('---')
print(yb_new)

tensor([[54, 59, 58],
        [53, 60, 43]])
---
tensor([[59, 58,  1],
        [60, 43, 12]])


In [14]:
# "Formula" -> Maximum value in input < num embeddings. Why?
"""
Scenario A: It works
    - Layer: nn.Embedding(65, 65)
        - Vocabulary Size = 65 (Valid indices are 0 to 64)
    - Input: Shape (3, 2). Let's assume the data inside is [[12, 50], [2, 5]]
    - Result: All numbers (12, 50, 2, 5) are less than 65. The lookup succeeds.

Scenario B: It fails
    - Layer: nn.Embedding(10, 10)
        - Vocabulary Size = 10 (Valid indices are 0 to 9)
    - Input: Same input [[12, 50], [2, 5]]
    - Result: The model tries to look up index 12 and 50, but it only has rows 0-9 in its table. It chrashes.
"""
emb = nn.Embedding(61, 61)

In [15]:
emb = nn.Embedding(xb_new.max()+1, xb_new.max()+1)
logis = emb(xb_new)
logits.shape

torch.Size([32, 65])

In [16]:
# create a Pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [17]:
batch_size = 32
for step in range(10000):

    # sample a batch of data
    xb, yb = get_batch('train')
    
    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.539217948913574


In [18]:
print(decode(model.generate(torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


Ay y-s har terewrulorvek.
Fve e ivatofor his wee, tanald; eaGounerntanke igro boumit
the.'sousthounker'd f ge t, IAboupa sers HEHefextwike,
Cis rnorusenks, s my qure. f he ify.
AReldotos, thenorelimperowe henserat he udeaveceaveandvVoo howed a w avee tul: ieveavinthod anofeaprthour thelim d, worinetse hast$forbupowirois gr mewhareckngote
ULO,
He whimetoley as come raralo l-ose t d s'serere mo nd ma t
we pin par d CHAY her s?qurDULO: ce'den
BRERa hadWPUNe


DWe atheng in st pha'BO:
HENonomangnnd 


### The Mathematical trick in self-attention

In [19]:
# consider the following toy example:

torch.manual_seed(1337)
B, T, C = 4, 8, 2   # batch, time, channels
x = torch.randn(B, T, C)
x

tensor([[[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]],

        [[ 1.3488, -0.1396],
         [ 0.2858,  0.9651],
         [-2.0371,  0.4931],
         [ 1.4870,  0.5910],
         [ 0.1260, -1.5627],
         [-1.1601, -0.3348],
         [ 0.4478, -0.8016],
         [ 1.5236,  2.5086]],

        [[-0.6631, -0.2513],
         [ 1.0101,  0.1215],
         [ 0.1584,  1.1340],
         [-1.1539, -0.2984],
         [-0.5075, -0.9239],
         [ 0.5467, -1.4948],
         [-1.2057,  0.5718],
         [-0.5974, -0.6937]],

        [[ 1.6455, -0.8030],
         [ 1.3514, -0.2759],
         [-1.5108,  2.1048],
         [ 2.7630, -1.7465],
         [ 1.4516, -1.5103],
         [ 0.8212, -0.2115],
         [ 0.7789,  1.5333],
         [ 1.6097, -0.4032]]])

In [20]:
# version 1
# We want x[b, t] = mean_{i<=t} x[b, i]
xbow = torch.zeros(B, T, C)
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b, t] = torch.mean(xprev, 0)

In [21]:
# version 2
wei = torch.tril(torch.ones(T, T))
wei = wei / torch.sum(wei, dim=1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) --> (B, T, C)

In [22]:
# version 3: use softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x 

In [23]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b 
print('a=')
print(a)
print('---')
print('b=')
print(b)
print('---')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
---
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
---
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [24]:
# consider the following toy example:

torch.manual_seed(1337)
B, T, C = 2, 3, 2   # batch, time, channels
x = torch.randn(B, T, C)
x

tensor([[[-2.0260, -2.0655],
         [-1.2054, -0.9122],
         [-1.2502,  0.8032]],

        [[-0.2071,  0.0544],
         [ 0.1378, -0.3889],
         [ 0.5133,  0.3319]]])

In [25]:
tril = torch.tril(torch.ones(T, T))
tril

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [26]:
wei = torch.zeros((T, T))
wei

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])

In [27]:
wei = wei.masked_fill(tril == 0, float('-inf'))
wei

tensor([[0., -inf, -inf],
        [0., 0., -inf],
        [0., 0., 0.]])

In [28]:
wei = F.softmax(wei, dim=-1)
wei

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])

In [29]:
x

tensor([[[-2.0260, -2.0655],
         [-1.2054, -0.9122],
         [-1.2502,  0.8032]],

        [[-0.2071,  0.0544],
         [ 0.1378, -0.3889],
         [ 0.5133,  0.3319]]])

In [30]:
wei @ x

tensor([[[-2.0260e+00, -2.0655e+00],
         [-1.6157e+00, -1.4889e+00],
         [-1.4939e+00, -7.2483e-01]],

        [[-2.0712e-01,  5.4416e-02],
         [-3.4641e-02, -1.6725e-01],
         [ 1.4801e-01, -8.5954e-04]]])

In [31]:
# version 4: self-attention!
torch.manual_seed(1337)
B, T, C = 4, 8, 32 # batch, time, channels
x = torch.randn(B, T, C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)      # (B, T, 16)
q = query(x)    # (B, T, 16)

wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) --> (B, T, T)

tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
# out = wei @ x # (B, T, T) @ (B, T, C) --> (B, T, C)

v = value(x)
out = wei @ v 
# out.shape 
out.shape

torch.Size([4, 8, 16])

In [32]:
out

tensor([[[-1.5713e-01,  8.8009e-01,  1.6152e-01, -7.8239e-01, -1.4289e-01,
           7.4676e-01,  1.0068e-01, -5.2395e-01, -8.8726e-01,  1.9068e-01,
           1.7616e-01, -5.9426e-01, -4.8124e-01, -4.8598e-01,  2.8623e-01,
           5.7099e-01],
         [ 6.7643e-01, -5.4770e-01, -2.4780e-01,  3.1430e-01, -1.2799e-01,
          -2.9521e-01, -4.2962e-01, -1.0891e-01, -4.9282e-02,  7.2679e-01,
           7.1296e-01, -1.1639e-01,  3.2665e-01,  3.4315e-01, -7.0975e-02,
           1.2716e+00],
         [ 4.8227e-01, -1.0688e-01, -4.0555e-01,  1.7696e-01,  1.5811e-01,
          -1.6967e-01,  1.6217e-02,  2.1509e-02, -2.4903e-01, -3.7725e-01,
           2.7867e-01,  1.6295e-01, -2.8951e-01, -6.7610e-02, -1.4162e-01,
           1.2194e+00],
         [ 1.9708e-01,  2.8561e-01, -1.3028e-01, -2.6552e-01,  6.6781e-02,
           1.9535e-01,  2.8073e-02, -2.4511e-01, -4.6466e-01,  6.9287e-02,
           1.5284e-01, -2.0324e-01, -2.4789e-01, -1.6213e-01,  1.9474e-01,
           7.6778e-01],
    

In [33]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [34]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [35]:
B, T, C = 4, 8, 16

In [36]:
a = torch.ones(B, T, C)
b = torch.ones(B, T, C)
a.shape, b.shape

(torch.Size([4, 8, 16]), torch.Size([4, 8, 16]))

In [37]:
a @ b.transpose(-1, -2)

tensor([[[16., 16., 16., 16., 16., 16., 16., 16.],
         [16., 16., 16., 16., 16., 16., 16., 16.],
         [16., 16., 16., 16., 16., 16., 16., 16.],
         [16., 16., 16., 16., 16., 16., 16., 16.],
         [16., 16., 16., 16., 16., 16., 16., 16.],
         [16., 16., 16., 16., 16., 16., 16., 16.],
         [16., 16., 16., 16., 16., 16., 16., 16.],
         [16., 16., 16., 16., 16., 16., 16., 16.]],

        [[16., 16., 16., 16., 16., 16., 16., 16.],
         [16., 16., 16., 16., 16., 16., 16., 16.],
         [16., 16., 16., 16., 16., 16., 16., 16.],
         [16., 16., 16., 16., 16., 16., 16., 16.],
         [16., 16., 16., 16., 16., 16., 16., 16.],
         [16., 16., 16., 16., 16., 16., 16., 16.],
         [16., 16., 16., 16., 16., 16., 16., 16.],
         [16., 16., 16., 16., 16., 16., 16., 16.]],

        [[16., 16., 16., 16., 16., 16., 16., 16.],
         [16., 16., 16., 16., 16., 16., 16., 16.],
         [16., 16., 16., 16., 16., 16., 16., 16.],
         [16., 16., 16., 16

In [38]:
b.transpose(-1, -2).shape

torch.Size([4, 16, 8])

In [39]:
a.shape

torch.Size([4, 8, 16])

In [40]:
# (4, 8, 16) @ (4, 16, 8) --> (4, 8, 8)

# (4, 8, 16)
# (4, 16, 8)
# |
# (4, 8, 8)

In [41]:
# version 4: self-attention! (cleaned)
torch.manual_seed(1337)
B, T, C = 4, 8, 32 # batch, time, channels
x = torch.randn(B, T, C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)      # (B, T, 16)
q = query(x)    # (B, T, 16)

wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) --> (B, T, T)

tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf')) # decoder block | delete this line, become encoder
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v 
# out = wei @ x 

out.shape

torch.Size([4, 8, 16])

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [42]:
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [43]:
k.var()

tensor(1.0449)

In [44]:
q.var()

tensor(1.0700)

In [45]:
wei.var()

tensor(1.0918)

In [46]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, 0.5]), dim=-1)

tensor([0.2245, 0.1663, 0.2742, 0.3349])

In [48]:
x = torch.tensor([0, 1, 2, 3, 4])
torch.save(x, 'tensor.pt')